In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import string
import sklearn
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import seaborn as sns

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble,calibration

#from sklearn.cross_validation import KFold
#from sklearn.cross_validation import train_test_split

#import pandas, numpy, textblob, string
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

train = pd.read_csv(r'training_matrix.csv')
valid = pd.read_csv(r'validating_matrix.csv')
test = pd.read_csv(r'testing_matrix.csv')

train.loc[train.label == 1, 'label'] = 0
train.loc[train.label == -1, 'label'] = 1

valid.loc[valid.label == 1, 'label'] = 0
valid.loc[valid.label == -1, 'label'] = 1

test.loc[test.label == 1, 'label'] = 0
test.loc[test.label == -1, 'label'] = 1


train['char_count'] = train['review'].apply(len)
train['word_count'] = train['review'].apply(lambda x: len(x.split()))
train['word_density'] = train['char_count'] / (train['word_count']+1)
train['punctuation_count'] = train['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
train['title_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
train['upper_case_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

valid['char_count'] = valid['review'].apply(len)
valid['word_count'] = valid['review'].apply(lambda x: len(x.split()))
valid['word_density'] = valid['char_count'] / (valid['word_count']+1)
valid['punctuation_count'] = valid['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
valid['title_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
valid['upper_case_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

test['char_count'] = test['review'].apply(len)
test['word_count'] = test['review'].apply(lambda x: len(x.split()))
test['word_density'] = test['char_count'] / (test['word_count']+1)
test['punctuation_count'] = test['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
test['title_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
test['upper_case_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Turker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# reviewer_centric features #
#train.user_id.value_counts()

train['date.1'] = pd.to_datetime(train['date.1'])
train['day_of_week'] = train['date.1'].dt.day_name()
#train.day_of_week.value_counts()[:10]]
dofw = train.day_of_week.unique()
for day in range(len(dofw)):
        label=train['label'][train.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

train['user_id_no_of_review'] = train.groupby('user_id')['user_id'].transform('size')
train['user_id_ave_rating'] = train.groupby('user_id')['rating'].transform('mean')
#train['user_id_std_rating'] = train.groupby('user_id')['rating'].transform('std')
train['user_id_ave_no_words'] = train.groupby('user_id')['word_count'].transform('mean')
train['user_id_max_review_a_day'] = train['user_id_no_of_review']
grouped = train.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#train[:][train.user_id==9236].sort_index(axis = 0)
    
valid['date.1'] = pd.to_datetime(valid['date.1'])
valid['day_of_week'] = valid['date.1'].dt.day_name()
#valid.day_of_week.value_counts()[:10]]
dofw = valid.day_of_week.unique()
for day in range(len(dofw)):
        label=valid['label'][valid.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

valid['user_id_no_of_review'] = valid.groupby('user_id')['user_id'].transform('size')
valid['user_id_ave_rating'] = valid.groupby('user_id')['rating'].transform('mean')
#valid['user_id_std_rating'] = valid.groupby('user_id')['rating'].transform('std')
valid['user_id_ave_no_words'] = valid.groupby('user_id')['word_count'].transform('mean')
valid['user_id_max_review_a_day'] = valid['user_id_no_of_review']
grouped = valid.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#valid[:][valid.user_id==9236].sort_index(axis = 0) 

test['date.1'] = pd.to_datetime(test['date.1'])
test['day_of_week'] = test['date.1'].dt.day_name()
#test.day_of_week.value_counts()[:10]]
dofw = test.day_of_week.unique()
for day in range(len(dofw)):
        label=test['label'][test.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

test['user_id_no_of_review'] = test.groupby('user_id')['user_id'].transform('size')
test['user_id_ave_rating'] = test.groupby('user_id')['rating'].transform('mean')
#test['user_id_std_rating'] = test.groupby('user_id')['rating'].transform('std')
test['user_id_ave_no_words'] = test.groupby('user_id')['word_count'].transform('mean')
test['user_id_max_review_a_day'] = test['user_id_no_of_review']
grouped = test.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#test[:][test.user_id==9236].sort_index(axis = 0)   

***********Day is  Thursday
fake is  764
real is  755
tot is  1519
Ratio is  50.29624753127058
len is  1519
***********Day is  Tuesday
fake is  800
real is  788
tot is  1588
Ratio is  50.377833753148614
len is  1588
***********Day is  Friday
fake is  758
real is  701
tot is  1459
Ratio is  51.953392734749826
len is  1459
***********Day is  Saturday
fake is  756
real is  744
tot is  1500
Ratio is  50.4
len is  1500
***********Day is  Monday
fake is  858
real is  913
tot is  1771
Ratio is  48.4472049689441
len is  1771
***********Day is  Sunday
fake is  790
real is  903
tot is  1693
Ratio is  46.66272888363851
len is  1693
***********Day is  Wednesday
fake is  836
real is  758
tot is  1594
Ratio is  52.446675031367626
len is  1594
***********Day is  Tuesday
fake is  150
real is  161
tot is  311
Ratio is  48.231511254019296
len is  311
***********Day is  Thursday
fake is  129
real is  128
tot is  257
Ratio is  50.19455252918288
len is  257
***********Day is  Saturday
fake is  137
real is 

In [3]:
# product_centric features #
#train.prod_id.value_counts()

train['prod_id_no_of_review'] = train.groupby('prod_id')['prod_id'].transform('size') + 1000.
train['prod_id_ave_rating'] = train.groupby('prod_id')['rating'].transform('mean') + 1000.
#train['prod_id_std_rating'] = train.groupby('prod_id')['rating'].transform('std') + 1000.
train['prod_id_ave_no_words'] = train.groupby('prod_id')['word_count'].transform('mean') + 1000.
train['prod_id_max_review_a_day'] = train['prod_id_no_of_review'] + 1000.
grouped = train.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#train[:][train.prod_id==9236].sort_index(axis = 0)

valid['prod_id_no_of_review'] = valid.groupby('prod_id')['prod_id'].transform('size') + 1000.
valid['prod_id_ave_rating'] = valid.groupby('prod_id')['rating'].transform('mean') + 1000.
#valid['prod_id_std_rating'] = valid.groupby('prod_id')['rating'].transform('std') + 1000.
valid['prod_id_ave_no_words'] = valid.groupby('prod_id')['word_count'].transform('mean') + 1000.
valid['prod_id_max_review_a_day'] = valid['prod_id_no_of_review']
grouped = valid.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#valid[:][valid.prod_id==9236].sort_index(axis = 0)  

test['prod_id_no_of_review'] = test.groupby('prod_id')['prod_id'].transform('size') + 1000.
test['prod_id_ave_rating'] = test.groupby('prod_id')['rating'].transform('mean') + 1000.
#test['prod_id_std_rating'] = test.groupby('prod_id')['rating'].transform('std') + 1000.
test['prod_id_ave_no_words'] = test.groupby('prod_id')['word_count'].transform('mean') + 1000.
test['prod_id_max_review_a_day'] = test['prod_id_no_of_review']
grouped = test.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#test[:][test.prod_id==9236].sort_index(axis = 0)  
                  

In [4]:
train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+ \
train.user_id_no_of_review.astype(str) + ' ' + train.user_id_ave_rating.astype(str) + ' '+\
train.user_id_ave_no_words.astype(str) + ' ' + train.user_id_max_review_a_day.astype(str) + ' '+\
train.prod_id_no_of_review.astype(str) + ' ' + train.prod_id_ave_rating.astype(str) + ' '+\
train.prod_id_ave_no_words.astype(str) + ' ' + train.prod_id_max_review_a_day.astype(str) + ' '+\
train.day_of_week.astype(str) + ' '+train.review  

#train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+  train.review
#test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+test.review

valid['total']=valid.user_id.astype(str) + ' '+ valid.prod_id.astype(str) + ' ' + valid.rating.astype(str) + ' '+ \
valid.user_id_no_of_review.astype(str) + ' ' + valid.user_id_ave_rating.astype(str) + ' '+\
valid.user_id_ave_no_words.astype(str) + ' ' + valid.user_id_max_review_a_day.astype(str) + ' '+\
valid.prod_id_no_of_review.astype(str) + ' ' + valid.prod_id_ave_rating.astype(str) + ' '+\
valid.prod_id_ave_no_words.astype(str) + ' ' + valid.prod_id_max_review_a_day.astype(str) + ' '+\
valid.day_of_week.astype(str) + ' '+ valid.review  

test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+ \
test.user_id_no_of_review.astype(str) + ' ' + test.user_id_ave_rating.astype(str) + ' '+\
test.user_id_ave_no_words.astype(str) + ' ' + test.user_id_max_review_a_day.astype(str) + ' '+\
test.prod_id_no_of_review.astype(str) + ' ' + test.prod_id_ave_rating.astype(str) + ' '+\
test.prod_id_ave_no_words.astype(str) + ' ' + test.prod_id_max_review_a_day.astype(str) + ' '+\
test.day_of_week.astype(str) + ' '+ test.review  

In [5]:
from nltk import SnowballStemmer
porter = SnowballStemmer("english")

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Turker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from sklearn.decomposition import PCA
from scipy import sparse
def apply_PCA(feature_vector_train, feature_vector_valid, feature_vector_test, n_components = 300):
    # applys PCA
    pca = PCA(n_components)
    #pca = PCA(0.95)
    pca.fit(feature_vector_train.toarray())
    xtrain = pca.transform(feature_vector_train.toarray())  
    xvalid = pca.transform(feature_vector_valid.toarray())       
    xtest  = pca.transform(feature_vector_test.toarray()) 
    
    feature_vector_train = sparse.csr_matrix(xtrain)
    feature_vector_valid = sparse.csr_matrix(xvalid)
    feature_vector_test  = sparse.csr_matrix(xtest)    
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]

from sklearn.decomposition import LatentDirichletAllocation
def apply_LDA(feature_vector_train, feature_vector_valid, feature_vector_test, n_components = 300):
    # applys lda
    lda = LatentDirichletAllocation(n_components, random_state=0)
    #lda = lda(0.95)
    lda.fit(feature_vector_train.toarray())
    xtrain = lda.transform(feature_vector_train.toarray())  
    xvalid = lda.transform(feature_vector_valid.toarray())       
    xtest  = lda.transform(feature_vector_test.toarray()) 
    
    feature_vector_train = sparse.csr_matrix(xtrain)
    feature_vector_valid = sparse.csr_matrix(xvalid)
    feature_vector_test  = sparse.csr_matrix(xtest)    
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]


def scale_feature(col):
    x = col.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()    
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    col = x_scaled
    return col




from scipy.sparse import hstack
def add_features(feature_vector_train, feature_vector_valid, feature_vector_test):
    
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_no_of_review'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_ave_rating'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_ave_no_words'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_max_review_a_day'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_no_of_review'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_ave_rating'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_ave_no_words'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_max_review_a_day'])[:,None]))    
    feature_vector_train = hstack((feature_vector_train,np.array(train['char_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['word_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['word_density'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['punctuation_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['title_word_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['upper_case_word_count'])[:,None]))
    
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_no_of_review'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_ave_rating'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_ave_no_words'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_max_review_a_day'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_no_of_review'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_ave_rating'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_ave_no_words'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_max_review_a_day'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['char_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['word_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['word_density'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['punctuation_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['title_word_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['upper_case_word_count'])[:,None]))
    
    
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_no_of_review'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_ave_rating'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_ave_no_words'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_max_review_a_day'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_no_of_review'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_ave_rating'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_ave_no_words'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_max_review_a_day'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['char_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['word_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['word_density'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['punctuation_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['title_word_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['upper_case_word_count'])[:,None]))
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]

In [7]:
# create a count vectorizer object 
#count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words='english')
#count_vect.fit(train.total)
#
#x_train_count =  count_vect.transform(train.total)
#x_valid_count =  count_vect.transform(valid.total)
#x_test_count =  count_vect.transform(test.total)

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words='english')
count_vect.fit(train.review)

x_train_count =  count_vect.transform(train.review)
x_valid_count =  count_vect.transform(valid.review)
x_test_count =  count_vect.transform(test.review)

count_vect_LR = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words='english')
count_vect_LR.fit(train.total)

x_train_count_LR =  count_vect_LR.transform(train.total)
x_valid_count_LR =  count_vect_LR.transform(valid.total)
x_test_count_LR =  count_vect_LR.transform(test.total)

[x_train_count_LR, x_valid_count_LR, x_test_count_LR] = apply_PCA(x_train_count, x_valid_count, x_test_count)
[x_train_count_LR, x_valid_count_LR, x_test_count_LR] = add_features(x_train_count_LR, x_valid_count_LR, x_test_count_LR)

In [8]:
from sklearn.model_selection import cross_val_score
def train_model_all(classifier, feature_vector_train, label_train, feature_vector_valid, label_valid, feature_vector_test, label_test, is_neural_net=False):
#    scoring = {'acc': 'accuracy',
#           'prec_macro': 'precision_macro',
#           'rec_micro': 'recall_macro'}
#    scores = cross_val_score(classifier,feature_vector_train, label_train, scoring=scoring, cv=5)
    scores = cross_val_score(classifier,feature_vector_train, label_train, cv=5)
    print(scores)
    # fit the train dataset on the classifier
    classifier.fit(feature_vector_train, label_train)
    # predict the labels on train and test dataset
    probs_train = classifier.predict_proba(feature_vector_train)
    probs_valid = classifier.predict_proba(feature_vector_valid)
    probs_test = classifier.predict_proba(feature_vector_test)
    #print("Probs is ",probs[:,1])
    # predict the labels on validation dataset    
    predictions_train = classifier.predict(feature_vector_train)
    predictions_valid = classifier.predict(feature_vector_valid)
    predictions_test = classifier.predict(feature_vector_test)
    return [predictions_train, np.mean(scores), probs_train , predictions_valid,metrics.accuracy_score(predictions_valid, label_valid),probs_valid,  predictions_test,metrics.accuracy_score(predictions_test, label_test),probs_test]

def train_model(classifier, feature_vector_train, label, feature_vector_valid,label_valid, is_neural_net=False):
    # fit the train dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    probs = classifier.predict_proba(feature_vector_valid)
    #print("Probs is ",probs[:,1])
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return [predictions,metrics.accuracy_score(predictions, label_valid),probs]

In [9]:
## Create first pipeline for base without reducing features.
#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
#
#pipe = Pipeline([('classifier' , linear_model.LogisticRegression())])
## pipe = Pipeline([('classifier', RandomForestClassifier())])
##lr = linear_model.LogisticRegression()
#
## Create param grid.
##solver_options = ['newton-cg', 'lbfgs', 'liblinear', 'sag']
#param_grid = [
#    {'classifier' : [linear_model.LogisticRegression()],
#     'classifier__penalty' : ['L1','l2'],
#    'classifier__C' : np.logspace(1, 2, 3),
#    'classifier__solver' : ['liblinear']}]
#
## Create grid search object
#
#clf = GridSearchCV(pipe, param_grid = param_grid,verbose=True, cv=5, n_jobs=-1)
#
## Fit on data
#
#best_clf = clf.fit(x_train_tfidf_ngram_chars, train.label)
#
#pd.DataFrame.from_dict(clf.cv_results_)

In [10]:
train_y=train.label
test_y=test.label
valid_y=valid.label

train_x=x_train_count
test_x=x_test_count
valid_x=x_valid_count

[pred_train,acc_train,prob_train, pred_valid,acc_valid,prob_valid, pred_test,acc_test,prob_test] =\
train_model_all(linear_model.LogisticRegression(), train_x, train_y, valid_x, valid_y, test_x, test_y)

print ("LR, Count Vectors => acc train: ", acc_train, "\tacc valid: ", acc_valid, "\tacc test: ", acc_test)
print("************ Logistic Regression Test***********")

[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(), train_x, train_y, test_x, test_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(test_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(test_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(test_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(test_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(test_y,probs_ll[:,1]))

print("************ Logistic Regression Valid***********")

[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(), train_x, train_y, valid_x, valid_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(valid_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(valid_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(valid_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(valid_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(valid_y,probs_ll[:,1]))

print("************ Logistic Regression Train***********")

[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(), train_x, train_y, train_x, train_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(train_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(train_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(train_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(train_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(train_y,probs_ll[:,1]))

C:\Users\Turker\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.61814915 0.61096137 0.61196043 0.61106115 0.62365108]
LR, Count Vectors => acc train:  0.6151566347999767 	acc valid:  0.6215 	acc test:  0.6142
************ Logistic Regression Test***********
LR, Count Vectors:  0.6142
Auc ROC is  0.6485118
Auc PRC is  0.6044357428407179
F1 score is 0.6435698447893571
Log loss is  0.8362353336338563
************ Logistic Regression Valid***********


C:\Users\Turker\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, Count Vectors:  0.6215
Auc ROC is  0.6599809999999999
Auc PRC is  0.6286268897010261
F1 score is 0.6447677146879399
Log loss is  0.8264354150509815
************ Logistic Regression Train***********


C:\Users\Turker\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, Count Vectors:  0.8553577849694355
Auc ROC is  0.9321586474252974
Auc PRC is  0.9261263422332073
F1 score is 0.8617341239150984
Log loss is  0.35870406860433984


In [11]:
train_x=x_train_count_LR
test_x=x_test_count_LR
valid_x=x_valid_count_LR


print("************ Logistic Regression Test + Features ***********")
[pred_train,acc_train,prob_train, pred_valid,acc_valid,prob_valid, pred_test,acc_test,prob_test] =\
train_model_all(linear_model.LogisticRegression(C=0.01, solver='liblinear'), train_x, train_y, valid_x, valid_y, test_x, test_y)

print ("LR, Count Vectors => acc train: ", acc_train, "\tacc valid: ", acc_valid, "\tacc test: ", acc_test)
[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(C=0.01, solver='liblinear'), train_x, train_y, test_x, test_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(test_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(test_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(test_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(test_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(test_y,probs_ll[:,1]))

recall_score = metrics.recall_score(test_y, predictions_ll)
print("recall_score score is",recall_score)
precision_score = metrics.precision_score(test_y, predictions_ll)
print("precision_score score is",precision_score)

results = pd.DataFrame()
results['prob'] = probs_ll[:,1]
results['predictions'] = predictions_ll
results['labels'] = test_y
results.to_csv(('LR.csv'))

print("************ Logistic Regression Valid***********")

[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(C=0.01, solver='liblinear'), train_x, train_y, valid_x, valid_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(valid_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(valid_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(valid_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(valid_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(valid_y,probs_ll[:,1]))

print("************ Logistic Regression Train***********")

[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(C=0.01, solver='liblinear'), train_x, train_y, train_x, train_y)
print ("LR, Count Vectors: ", accuracy_ll)
fpr_ll, tpr_ll, thresholds = metrics.roc_curve(train_y, probs_ll[:,1])
auc_ll = metrics.roc_auc_score(train_y, probs_ll[:,1])
print("Auc ROC is ",auc_ll)
precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(train_y, probs_ll[:,1])
auc_ll = metrics.auc(recall_ll, precision_ll)
print("Auc PRC is ",auc_ll)
f1_ll = metrics.f1_score(train_y, predictions_ll)
print("F1 score is",f1_ll)
print("Log loss is ",metrics.log_loss(train_y,probs_ll[:,1]))
## in 28

************ Logistic Regression Test + Features ***********
[0.68733154 0.655885   0.66636691 0.66771583 0.68255396]
LR, Count Vectors => acc train:  0.6719706445086518 	acc valid:  0.654 	acc test:  0.6537
LR, Count Vectors:  0.6537
Auc ROC is  0.71780496
Auc PRC is  0.7060661580298724
F1 score is 0.6850386539336062
Log loss is  0.6143693375427698
recall_score score is 0.7532
precision_score score is 0.6281901584653878
************ Logistic Regression Valid***********
LR, Count Vectors:  0.654
Auc ROC is  0.7086490000000001
Auc PRC is  0.6903775102407947
F1 score is 0.6828597616865261
Log loss is  0.6222454156814841
************ Logistic Regression Train***********
LR, Count Vectors:  0.6947141316073355
Auc ROC is  0.7649571804150551
Auc PRC is  0.7569854627997736
F1 score is 0.7084978540772533
Log loss is  0.5809369056318081


In [12]:
#features=['user_id','prod_id','rating']
#test=np.array(test[features])
#valid_y=test.label
#[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
##test=np.concatenate((test,np.resize(probs,(len(probs[:,1]),1))),1)
#
#train=np.array(train[features])
#valid_y=train.label
#[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xtrain_tfidf_ngram)
#train=np.concatenate((train,np.resize(probs,(len(probs[:,1]),1))),1)

## PLOTS
#import matplotlib.pyplot as plt
## plot the roc curve for the model
#plt.plot(fpr, tpr, linestyle='--', label='NB')
#plt.plot(fpr_ll, tpr_ll, linestyle='-', label='Logistic Regression')
##pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
## axis labels
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
## show the legend
#plt.legend()
## show the plot
#plt.show()
#
#import matplotlib.pyplot as plt
## plot the roc curve for the model
#plt.plot(recall, precision, marker='.',label='NB')
#plt.plot(recall_ll, precision_ll, marker='x',label='Logistic Regression')
##pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
## axis labels
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
## show the legend
#plt.legend()
## show the plot
#plt.show()
#
#[y,x]=calibration.calibration_curve(valid_y,probs[:,1])
#[y_ll,x_ll]=calibration.calibration_curve(valid_y,probs_ll[:,1])
#plt.plot(x,y, marker='o', linewidth=1, label='NB')
#plt.plot(x_ll,y_ll, marker='x', linewidth=1, label='Logistic Regression')
#x1=np.linspace(0,1,11)
#y1=x1
#plt.plot(x1,y1,linestyle='--')
#plt.legend()
#
#plt.hist(probs[:,1],bins=10,histtype='step',label='NB')
#plt.hist(probs_ll[:,1],bins=10,histtype='step',label='Logistic Regression')
#plt.xlabel('Prob')
#plt.ylabel('Count')
#plt.legend()
#
#y_test=test.label
#y_pred=predictions
#class_names=["True","False"]
#cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
#fig, ax = plt.subplots()
#tick_marks = np.arange(len(class_names))
#plt.xticks(tick_marks, class_names)
#plt.yticks(tick_marks, class_names)
#sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="viridis" ,fmt='g')
#ax.xaxis.set_label_position("top")
#plt.tight_layout()
#plt.title('Confusion matrix - NB', y=1.1)
#plt.ylabel('Actual label')
#plt.xlabel('Predicted label')
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print("Precision:",metrics.precision_score(y_test, y_pred))
#print("Recall:",metrics.recall_score(y_test, y_pred))
#
#
#y_test=test.label
#y_pred=predictions_ll
#class_names=["True","False"]
#cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
#fig, ax = plt.subplots()
#tick_marks = np.arange(len(class_names))
#plt.xticks(tick_marks, class_names)
#plt.yticks(tick_marks, class_names)
#sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="viridis" ,fmt='g')
#ax.xaxis.set_label_position("top")
#plt.tight_layout()
#plt.title('Confusion matrix - Logistic Regression', y=1.1)
#plt.ylabel('Actual label')
#plt.xlabel('Predicted label')
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print("Precision:",metrics.precision_score(y_test, y_pred))
#print("Recall:",metrics.recall_score(y_test, y_pred))

#import pandas as pd
#from sklearn import preprocessing
#x = train[['rating']].values.astype(float)
## Create a minimum and maximum processor object
#min_max_scaler = preprocessing.MinMaxScaler()
#
## Create an object to transform the data to fit minmax processor
#x_scaled = min_max_scaler.fit_transform(x)
#train.rating=x_scaled

# Trainind data set results
#train_y=train.label
#valid_y=train.label
#
#train_x=xtrain_tfidf_ngram
#test_x=xtrain_tfidf_ngram
#
#print("************ Naive Bayes***********")
#[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, test_x)
#print ("NB, Count Vectors: ", accuracy)
#fpr, tpr, thresholds = metrics.roc_curve(valid_y, probs[:,1])
#auc = metrics.roc_auc_score(valid_y, probs[:,1])
#print("Auc ROC is ",auc)
#precision, recall, thresholds = metrics.precision_recall_curve(valid_y, probs[:,1])
#auc = metrics.auc(recall, precision)
#print("Auc PRC is ",auc)
#f1 = metrics.f1_score(valid_y, predictions)
#print("F1 score is",f1)
#print("Log loss is ",metrics.log_loss(valid_y,probs[:,1]))
#
#
#
#print("************ Logistic Regression***********")
#
#[predictions_ll,accuracy_ll,probs_ll] = train_model(linear_model.LogisticRegression(), train_x, train_y, test_x)
#print ("LR, Count Vectors: ", accuracy_ll)
#fpr_ll, tpr_ll, thresholds = metrics.roc_curve(valid_y, probs_ll[:,1])
#auc_ll = metrics.roc_auc_score(valid_y, probs_ll[:,1])
#print("Auc ROC is ",auc_ll)
#precision_ll, recall_ll, thresholds = metrics.precision_recall_curve(valid_y, probs_ll[:,1])
#auc_ll = metrics.auc(recall_ll, precision_ll)
#print("Auc PRC is ",auc_ll)
#f1_ll = metrics.f1_score(valid_y, predictions_ll)
#print("F1 score is",f1_ll)
#print("Log loss is ",metrics.log_loss(valid_y,probs_ll[:,1]))